In [92]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import transforms
import numpy as np
from torchsummary import summary as summary_

In [104]:
class selfModule(nn.Module):
    def __init__(self):
        super(selfModule, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1, stride=2),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1, stride=2),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(256,200),
            nn.Tanh(),
            nn.Linear(200,10),
            nn.LogSoftmax()
        )
        
    def forward(self, x):
        out = self.model(x)
        return out

In [105]:
model = selfModule()
summary_(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 14, 14]             320
              Tanh-2           [-1, 32, 14, 14]               0
         MaxPool2d-3             [-1, 32, 7, 7]               0
            Conv2d-4             [-1, 64, 4, 4]          18,496
              Tanh-5             [-1, 64, 4, 4]               0
         MaxPool2d-6             [-1, 64, 2, 2]               0
           Flatten-7                  [-1, 256]               0
            Linear-8                  [-1, 200]          51,400
              Tanh-9                  [-1, 200]               0
           Linear-10                   [-1, 10]           2,010
       LogSoftmax-11                   [-1, 10]               0
Total params: 72,226
Trainable params: 72,226
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [107]:
model.state_dict()

OrderedDict([('model.0.weight',
              tensor([[[[ 6.9470e-02, -3.2027e-01,  6.9798e-02],
                        [ 1.5328e-01,  2.3020e-01,  1.2463e-01],
                        [-2.5464e-02, -1.7015e-01, -6.7813e-02]]],
              
              
                      [[[ 1.9467e-02,  1.3905e-01,  8.0599e-02],
                        [ 1.8126e-01, -1.2514e-01, -8.9499e-02],
                        [-3.2010e-01,  3.2651e-03,  9.0117e-02]]],
              
              
                      [[[-2.3808e-01, -3.1156e-01, -1.0573e-01],
                        [-2.0043e-01,  2.5320e-01,  8.8500e-02],
                        [-9.1475e-02, -2.7554e-01, -1.6991e-01]]],
              
              
                      [[[ 2.6915e-01, -7.9201e-02, -3.2234e-01],
                        [-5.9149e-03,  1.3894e-01,  1.0786e-01],
                        [ 1.8040e-01,  3.6466e-02,  2.9750e-01]]],
              
              
                      [[[-1.0096e-01, -1.3276e-01, -1.9667e-

In [123]:
a= torch.Tensor([[2,1,3],[2,0,4]])

class Parameter():
  def __init__(self):
    self.

torch.Size([2, 3])

In [117]:
def my_min(sequence, key_func=None):
    """
    Return the maximum element of a sequence.
    key_func is an optional one-argument ordering function.
    """
    if not sequence:
        raise ValueError('empty sequence')

    if not key_func:
        key_func = identity

    minimum = sequence[0]

    for item in sequence:
        # Ask the key func which property to compare
        if key_func(item) < key_func(minimum):
            minimum = item

    return minimum

def weight(x):
    return x[0]

def plus_minus(x):
  return 1 if x > 0 else -1

In [10]:
def update_grad(update_info, global_params):
  for grad in update_info:
    if len(grad[2]) == 1:
      global_params[grad[1]][grad[2][0]] += grad[0]*grad[3]
    else:
      global_params[grad[1]][grad[2][0]][grad[2][1]] += grad[0]*grad[3]

In [109]:
def top_N_param_grad(param_grad, N):
  abs_top_N = []

  for i in range(N):
    for j in range(len(param_grad['model.0.weight'][i])):
      abs_top_N.append([abs(param_grad['model.0.weight'][i][j]), 'model.0.weight', [i,j], plus_minus(param_grad['model.0.weight'][i][j])])

  minVal = min(abs_top_N)
  for key in param_grad.keys():
    for i in range(len(param_grad[key])):
      if 'weight' in key:
        for j in range(len(param_grad[key][i])):
          if minVal[0] > abs(param_grad[key][i][j]):
            abs_top_N.remove(minVal)
            abs_top_N.append([abs(param_grad[key][i][j]), key, [i,j], plus_minus(param_grad[key][i][j])])
            minVal = min(abs_top_N)
      elif minVal[0] > abs(param_grad[key][i]):
        abs_top_N.remove(minVal)
        abs_top_N.append([abs(param_grad[key][i]), key, [i], plus_minus(param_grad[key][i])])
        minVal = min(abs_top_N)

  return abs_top_N

In [68]:
theta_u = 0.1
threadLock = threading.Lock()
exitFlag = 0


class parameterServer():
  def __init__(self, initial_params):
    self.globalParams = initial_params.copy()
    self.update_infos = []

  def upload(self, update_info):
    self.update_infos.append(update_info)
  
  def update(self):
    print("server updating")
    for info in self.update_infos:
      update_grad(info, self.globalParams)
    self.update_info = []

class edgeDevice():
  def __init__(self, deviceID, server, initial_params, dataloader):
    self.deviceID = deviceID
    self.module = selfModule()
    self.params = initial_params.copy()
    self.server = server
    self.dataloader = dataloader
  
  def train(self):
      print(self.deviceID + " ttraining")

      self.params = self.server.globalParams.copy()
      self.module.load_state_dict(self.params)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.SGD(self.module.parameters(), lr=0.01)

      self.module.train()
      train_loss = 0
      total = 0
      correct = 0

      print(len(self.dataloader))

      for batch_idx, data in enumerate(self.dataloader):
          image, label = data
          # Grad initialization
          optimizer.zero_grad()
          # Forward propagation
          output = self.module(image)
          # Calculate loss
          loss = criterion(output, label)
          # Backprop
          loss.backward()
          # Weight update
          optimizer.step()
          
          train_loss += loss.item()

          _,predicted = output.max(1)
          total += label.size(0)
          correct += predicted.eq(label).sum().item()
        
          if (batch_idx+1) % 1000 == 0:
            print("Step: {}/{} | train_loss: {:.4f} | Acc:{:.3f}%".format(batch_idx+1, len(self.dataloader), train_loss/1000, 100.*correct/total))

      param_grads = {key: self.module.state_dict()[key] - self.params[key] for key in self.params.keys()}
      top_grads = top_N_param_grad(param_grads,3)
      self.server.upload(top_grads)

In [69]:
device_num = 4
batch_size = 2

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

traindata_split = torch.utils.data.random_split(trainset, [int(trainset.data.shape[0] / device_num) for _ in range(device_num)])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]


testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)

# Download mnist data in ./data folder

In [75]:
for batch_idx, data in enumerate(train_loader[0]):
  image, label = data
  print(image.size())
  if batch_idx == 2:
    break

torch.Size([2, 1, 28, 28])
torch.Size([2, 1, 28, 28])
torch.Size([2, 1, 28, 28])


In [70]:
len(trainloader)

6000

In [110]:
num_of_device_in_round = 2

initialmodel = selfModule()

initial_params = initialmodel.state_dict()

server = parameterServer(initial_params)

devices = []

for i in range(device_num):
  devices.append(edgeDevice('device-'+str(i), server, initial_params, train_loader[i]))

for i in range(5):
  devices_selected = np.random.permutation(np.arange(device_num))[:num_of_device_in_round]
  print(devices_selected)
  for de_num in devices_selected:
    devices[de_num].train()
  server.update()

[1 3]
device-1 ttraining
7500


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Step: 1000/7500 | train_loss: 1.9822 | Acc:38.850%
Step: 2000/7500 | train_loss: 2.8601 | Acc:56.625%
Step: 3000/7500 | train_loss: 3.3720 | Acc:65.867%
Step: 4000/7500 | train_loss: 3.7704 | Acc:71.450%
Step: 5000/7500 | train_loss: 4.0803 | Acc:75.380%
Step: 6000/7500 | train_loss: 4.3601 | Acc:78.042%
Step: 7000/7500 | train_loss: 4.6066 | Acc:80.129%


RuntimeError: ignored

In [ ]:
import Queue
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, q, device):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.q = q
      self.device = device
   def run(self):
      print "Starting " + self.name
      process(self.name, self.q)
      print "Exiting " + self.name

def process_data(threadName, q):
   while not exitFlag:
      queueLock.acquire()
         if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print "%s processing %s" % (threadName, data)
         else:
            queueLock.release()
         time.sleep(1)